In [ ]:
import tensorflow as tf
import numpy as np 
import random

import gym

# Experience Replay buffer

In [ ]:
from numpy import dtype


class Buffer:
    
    def __init__(self):
        self.storage = np.empty(0)
        self.size = self.storage.size





    def save(self, data):

        # Free thousand slots in buffer
        while self.storage.size > 9000:
            self.storage = np.delete(self.storage, -1)


        self.storage = np.append(self.storage, data)
        self.storage = np.reshape(self.storage, (-1, 3))


        #return self.storage[0]

    def sample(self):
        return(tf.convert_to_tensor(self.storage[random.randint(0, self.size)], dtype=float))

    def lol(self):
        print(self.storage[0])
    




In [ ]:
a = np.array([1,2,3,4])

print(a)

print(type(a.size))

# DQN Network

In [ ]:
# Hyperparameter


In [ ]:
class Q_network(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.l_1 = tf.keras.layers.Dense(128,activation='relu')
    self.l_2 = tf.keras.layers.Dense(128,activation='relu')
    self.out = tf.keras.layers.Dense(4, activation='softmax')

  def call(self, input_data):
    x = self.appr(input_data)
    action = self.out(x)
    return action

In [ ]:
q_network = Q_network()
buffer = Buffer()
env = gym.make('LunarLander-v2')
epsilon = 0.02
gamma = 0.01
optimizer = tf.keras.optimizers.adam



for n in range(50):

        observation = env.reset()

        for step in range(100):
            # Starts environment again
            #env.render()

            # Choose random action of complete action space
            action = env.action_space.sample()
            # Do action and save information about its effect
            observation, reward, done, _ = env.step(action)

            buffer.save(np.array([observation, reward, done], dtype=object))

            if done:
                #print("Episode finished after {} timesteps".format(step + 1))
                break
        env.close()

        with tf.GradientTape() as tape:
                # Obtain q-value prediction

                q_values_pred = q_network(buffer.sample())
                #print(q_values_pred)

                # Epsilon greedy
                sample_epsilon = np.random.rand()
                
                # Random action
                if sample_epsilon <= epsilon:
                        action = np.random.choice(4)

                else:
                        action = np.argmax(q_values_pred[0])

        # Get reward still missing
        reward = get_reward(state, action)

        #
        q_value = q_values_pred[action]

        # Get state still missing
        next_state = get_state(state, action)

        # Next state == terminal state
        if next_state == False:
                next_q_value = 0 # No Q-value for terminal
        else:
                next_q_values = tf.stop_gradient(q_network(next_state)) # No gradient computation
                next_action = np.argmax(next_q_values[0])
                next_q_value = next_q_values[0, next_action]

        observed_q_value = reward + (gamma * next_q_value)

        # MSE
        loss_value = (observed_q_value-q_value)**2

        # Compute gradients
        grads = tape.gradient(loss_value, q_network.trainable_variables)

        "Apply gradients to update network weights"
        optimizer.apply_gradients(zip(grads, q_network.trainable_variables))

        "Update state"
        state = next_state






    
    

